In [19]:
# importamos las librerias necesarias como nltk para los stopwords
import numpy as np
import pandas as pd
import re
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from numpy.linalg import norm

# Carga y procesamiento de datos

In [20]:
#cargamos el dataset
df_reviews = pd.read_csv('/kaggle/input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_critic_reviews.csv')
df_movies = pd.read_csv('/kaggle/input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_movies.csv')

In [21]:
df_reviews

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...
...,...,...,...,...,...,...,...,...
1130012,m/zulu_dawn,Chuck O'Leary,False,Fantastica Daily,Rotten,2/5,2005-11-02,NaN
1130013,m/zulu_dawn,Ken Hanke,False,"Mountain Xpress (Asheville, NC)",Fresh,3.5/5,2007-03-07,"Seen today, it's not only a startling indictme..."
1130014,m/zulu_dawn,Dennis Schwartz,False,Dennis Schwartz Movie Reviews,Fresh,B+,2010-09-16,A rousing visual spectacle that's a prequel of...
1130015,m/zulu_dawn,Christopher Lloyd,False,Sarasota Herald-Tribune,Rotten,3.5/5,2011-02-28,"A simple two-act story: Prelude to war, and th..."


In [22]:
df_movies

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17707,m/zoot_suit,Zoot Suit,Mexican-American gangster Henry Reyna (Daniel ...,NaN,R,"Drama, Musical & Performing Arts",Luis Valdez,Luis Valdez,"Daniel Valdez, Edward James Olmos, Charles Aid...",1981-10-02,...,MCA Universal Home Video,Rotten,56.0,9.0,Upright,74.0,1195.0,2,5,4
17708,m/zootopia,Zootopia,From the largest elephant to the smallest shre...,The brilliantly well-rounded Zootopia offers a...,PG,"Action & Adventure, Animation, Comedy","Byron Howard, Rich Moore, Jared Bush","Jared Bush, Phil Johnston","J.K. Simmons, Kristen Bell, Octavia Spencer, A...",2016-03-04,...,Walt Disney Animation Studios,Certified-Fresh,98.0,291.0,Upright,92.0,101511.0,50,285,7
17709,m/zorba_the_greek,Zorba the Greek,Traveling to inspect an abandoned mine his fat...,NaN,NR,"Action & Adventure, Art House & International,...",NaN,NaN,"Anthony Quinn, Alan Bates, Irene Papas, Lila K...",1964-12-17,...,Fox,Fresh,80.0,10.0,Upright,86.0,7146.0,0,8,2
17710,m/zulu,Zulu,"In 1879, the Zulu nation hands colonial Britis...",Zulu patiently establishes a cast of colorful ...,PG,"Classics, Drama","Cy Endfield, Cyril Endfield","Cy Endfield, John Prebble","Stanley Baker, Jack Hawkins, Ulla Jacobsson, J...",1964-06-17,...,Paramount Pictures,Fresh,96.0,23.0,Upright,91.0,30193.0,6,22,1


In [49]:
# Usamos 'rotten_tomatoes_link' como la llave común entre ambas tablas.
df_reviews['review_content'] = df_reviews['review_content'].fillna('')

reviews_agrupadas = df_reviews.groupby('rotten_tomatoes_link')['review_content'].apply(lambda x: ' '.join(x))

# Unimos la info de películas con sus críticas
df_full = pd.merge(df_movies, reviews_agrupadas, on='rotten_tomatoes_link', how='left')

# Creamos el documento completo (Título + Género + Sinopsis + Críticas)
# Llenamos nulos con vacío "" para evitar errores
df_full['movie_content'] = (
    df_full['movie_title'].fillna('') + " " + 
    df_full['genres'].fillna('') + " " + 
    df_full['movie_info'].fillna('') + " " +
    df_full['review_content'].fillna('') 
)

In [56]:
#Funcion para procesar el texto y dejarlo "limpio"
stop_words = set(stopwords.words('english'))
def procesar_texto(texto):
    # transforma a minusculas y elimina signos especiales
    text = texto.lower()
    text = re.sub(pattern=r"<.*?>", repl='', string=text)
    text = re.sub(r"[^a-z0-9\s]", '', text) 
    text = re.sub(r'\s+', ' ', text).strip()

    #dividimos el texto en tokens
    tokens = text.split()
    #eliminamos las stopwords
    tokens_filtrados = [t for t in tokens if t not in stop_words and len(t) > 2]
    
    return tokens_filtrados


In [57]:
# aplicamos la función de procesar datos al corpus original y le transformamos a una lista ordenada 
corpus = df_full['movie_content'].apply(procesar_texto).tolist()

In [51]:
corpus[1]

['please',
 'give',
 'comedy',
 'kate',
 'catherine',
 'keener',
 'husband',
 'alex',
 'oliver',
 'platt',
 'wealthy',
 'new',
 'yorkers',
 'prowl',
 'estate',
 'sales',
 'make',
 'tidy',
 'profit',
 'reselling',
 'items',
 'bought',
 'cheaply',
 'buy',
 'apartment',
 'next',
 'door',
 'plan',
 'remodel',
 'soon',
 'current',
 'occupant',
 'cranky',
 'old',
 'woman',
 'dies',
 'kate',
 'troubled',
 'way',
 'husband',
 'earn',
 'living',
 'tries',
 'assuage',
 'guilt',
 'befriending',
 'tenant',
 'womans',
 'granddaughters',
 'overtures',
 'lead',
 'unexpected',
 'consequences',
 'like',
 'holofceners',
 'previous',
 'pictures',
 'please',
 'give',
 'derives',
 'narrative',
 'energy',
 'less',
 'series',
 'plotted',
 'incidents',
 'keenly',
 'observed',
 'interplay',
 'clashing',
 'personality',
 'tics',
 'worldviews',
 'muted',
 'almost',
 'rohmerlike',
 'moral',
 'tale',
 'doesnt',
 'quite',
 'dive',
 'deeply',
 'enough',
 'several',
 'characters',
 'keener',
 'peet',
 'hall',
 'shine

# Contrucción del sistema BM25

In [52]:
# obtenemos las estadisticas que usamos en la formula de bm25 que son el tamaño de cada documento
# y el promedio del tamaño de los documentos del corpus
def obtener_estadisticas_bm25(corpus):
    doc_lengths = [len(doc) for doc in corpus]
    avgdl = sum(doc_lengths) / len(corpus)
    return doc_lengths, avgdl

In [53]:
# calculamos idf para bm25
def calcular_idf_bm25(corpus):
    N = len(corpus)
    idf_dict = {}
    df_counts = Counter()
    for doc in corpus:
        df_counts.update(set(doc))
    
    for palabra, df in df_counts.items():
        idf_dict[palabra] = math.log((N - df + 0.5) / (df + 0.5) + 1)
    return idf_dict

In [58]:
# funcion que calcula el score para cada documento 
# ademas asignamos un valor de k1=1.5 para que la repetición de palabras no inflara el score
# b = 0.75 para que los documentos extensos no dominen frente a otros documentos con menos palabras
def score_bm25_documento(query_tokens, doc_tokens, doc_len, avgdl, idf_dict, k1=1.5, b=0.75):
    score = 0
    doc_counts = Counter(doc_tokens)
    
    for token in query_tokens:
        if token in doc_counts:
            tf = doc_counts[token]
            idf = idf_dict.get(token, 0)
            numerador = tf * (k1 + 1)
            denominador = tf + k1 * (1 - b + b * (doc_len / avgdl))
            score += idf * (numerador / denominador)
    return score

In [48]:
def buscar_bm25(consulta, corpus, top_n=5, k1=1.5, b=0.75):
    # Procesamos consulta y la transformamos a tokens
    query_tokens = procesar_texto(consulta)
    
    # obtenemos las estadisticas para bm25
    doc_lengths, avgdl = obtener_estadisticas_bm25(corpus)
    idf_bm25 = calcular_idf_bm25(corpus)
    
    # Calculo de los scores para todos los documentos
    scores = []
    for i, doc_tokens in enumerate(corpus):
        puntos = score_bm25_documento(
            query_tokens, doc_tokens, doc_lengths[i], avgdl, idf_bm25, k1, b
        )
        if puntos > 0:
            # Aquí guardamos id, puntos
            scores.append((i, puntos))

    ranking = sorted(scores, key=lambda x: x[1], reverse=True)[:top_n]
    
    return ranking

# funcon que permite realizar la busqueda pasandole la consulta e imprime los resultados con los valores de bm25
# ademas muestra información como el titulo, genero y la sipnosis para visualizar si los resultados son correctos
def motor_de_busqueda(consulta):
    print(f"Buscando: '{consulta}'")

    ranking = buscar_bm25(consulta, corpus)
    
    resultados = []

    for doc_id, score in ranking:
        
        # Recuperamos los datos originales usando el ID
        pelicula = df_full.iloc[doc_id]
        
        resultados.append({
            'Título': pelicula['movie_title'],
            'Score BM25': round(score, 4),
            'Género': pelicula['genres'],
            # Convertimos a string por seguridad en caso de nulos
            'Sinopsis': str(pelicula['movie_info'])[:150] + "..." 
        })
        
    if not resultados:
        return "No se encontraron coincidencias."
        
    return pd.DataFrame(resultados)

# Pruebas y resultados

In [54]:
print("\n--- RESULTADOS PARA 'VIAJES ESPACIALES' ---")
display(motor_de_busqueda("Space travel movies"))


--- RESULTADOS PARA 'VIAJES ESPACIALES' ---
Buscando: 'Space travel movies'


,Título,Score BM25,Género,Sinopsis
0,Hubble 3D,12.3587,"Documentary, Drama","In May 2009, NASA astronauts embark on a missi..."
1,Futurama: Bender's Big Score,12.0163,"Animation, Comedy, Science Fiction & Fantasy",Alien invaders force Bender (John DiMaggio) th...
2,It! The Terror From Beyond Space,11.5168,"Classics, Horror, Science Fiction & Fantasy",It's the future -- 1973 -- and a rescue team i...
3,Fly Me to the Moon,11.3651,"Animation, Kids & Family, Science Fiction & Fa...",Nat the fly loves listening to his grandfather...
4,Somewhere in Time,11.3624,"Drama, Science Fiction & Fantasy, Romance","In 1972, playwright Richard Collier (Christoph..."


In [55]:
print("\n--- RESULTADOS PARA 'PELICULAS FAMILIARES' ---")
display(motor_de_busqueda("Family movies for kids"))


--- RESULTADOS PARA 'PELICULAS FAMILIARES' ---
Buscando: 'Family movies for kids'


,Título,Score BM25,Género,Sinopsis
0,Spy Kids,8.7466,"Action & Adventure, Comedy, Kids & Family, Mys...",Two young kids become spies in attempt to save...
1,Snowmen,8.5936,"Drama, Kids & Family",A 10-year-old boy and his friends perform peri...
2,The Kids Are All Right,8.4686,"Comedy, Drama",Lesbian couple Jules (Julianne Moore) and Nic ...
3,Oddball and the Penguins (Oddball),8.4059,"Action & Adventure, Art House & International,...",A chicken farmer and his granddaughter train a...
4,Spy Kids 2: The Island of Lost Dreams,8.3152,"Action & Adventure, Kids & Family, Mystery & S...","Now full fledged Spy Kids, Carmen (Alexa Vega)..."


In [59]:
print("\n--- RESULTADOS PARA BUSQUEDA EXTRA ---")
display(motor_de_busqueda("Action movies in space"))


--- RESULTADOS PARA BUSQUEDA EXTRA' ' ---
Buscando: 'Action movies in space'


,Título,Score BM25,Género,Sinopsis
0,Lost in Space,8.9624,"Action & Adventure, Science Fiction & Fantasy",The space family Robinson is on a journey to A...
1,Space Cowboys,8.9566,"Action & Adventure, Comedy, Drama, Science Fic...","Clint Eastwood, Tommy Lee Jones, Donald Suther..."
2,Ôritsu uchûgun Oneamisu no tsubasa (Royal Spac...,8.9413,"Animation, Anime & Manga, Drama, Science Ficti...","In this animated film, Shiro (Leo Morimoto) fa..."
3,Space Chimps,8.8707,"Action & Adventure, Animation, Comedy, Kids & ...",Ham III (Andy Samberg) is the grandson of the ...
4,Zathura: A Space Adventure,8.8207,"Action & Adventure, Kids & Family, Science Fic...",After their father (Tim Robbins) is called int...


# Análisis de resultados

## Evaluación de la Consulta 1: "Space travel movies"
Para esta consulta, el sistema BM25 mostró una alta precisión, devolviendo documentos con scores de relevancia entre 11.36 y 12.35.

El sistema recuperó desde animación (Futurama) hasta terror clásico (It! The Terror From Beyond Space). Esto indica que el buscador no está sesgado hacia un solo género, sino que rastrea el concepto "viaje espacial" a través de todo el corpus.

La película Somewhere in Time (Puesto 5) trata sobre viajes en el tiempo, no espaciales. Este es un caso interesante de coincidencia parcial, el algoritmo encontró las palabras "travel" y "movies" con alta frecuencia, pero carece de la comprensión semántica para distinguir "tiempo" de "espacio".

## Evaluación de la Consulta 2: "Family movies for kids"
Los scores para esta consulta fueron ligeramente más bajos (rango 8.31 - 8.74), lo que indica que los términos son más comunes en la colección y por tanto tienen un IDF menor.

El Top 1 y Top 5 son películas de la saga "Spy Kids". Aquí el sistema funcionó a la perfección: coincidió la palabra "Kids" en el título, el género "Kids & Family" y seguramente múltiples menciones en las críticas.

El resultado #3 es "The Kids Are All Right". Aunque el algoritmo la trajo por contener las palabras "Kids" y probablemente "Family", esta es una película de Drama/Comedia orientada a adultos, no una película infantil.

## Validación del Modelo BM25 y la Fusión de Datos

La implementación de BM25 con parámetros $k1=1.5$ y $b=0.75$ fue crucial para el éxito del sistema frente a un modelo TF-IDF tradicional, fusionar todas las críticas (critic_reviews) con la sinopsis, creamos documentos de longitudes extremas. Sin la normalización de BM25, las películas populares con miles de palabras hubieran dominado el ranking simplemente por probabilidad. El parámetro $b$ corrigió este sesgo. En las críticas fusionadas, palabras como "movie" o "story" se repiten cientos de veces. La saturación de BM25 evitó que estos términos comunes inflaran artificialmente los puntajes.
## Propuesta de Mejoras Futuras 

Para solucionar el error de "The Kids Are All Right", se debería incorporar un filtro para que coincida con la busqueda.
Para la búsqueda de "Space travel", el sistema podría no encontrar películas que solo usen términos como "Interstellar" o "Orbit" sin decir explícitamente "travel".
Dado que la base de datos está en inglés, el sistema actual requiere consultas en inglés. Se podria implementar una herramienta que permita escribir busquedas en español e ingles.